<a href="https://colab.research.google.com/github/Ehtisham1053/Classification-ML-algorithms/blob/main/Logistic_regression_GUI_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

class LogisticRegressionTool:
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.model = None

    def plot_data(self, X, y):
        plt.figure(figsize=(6, 4))
        plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
        plt.title("Dataset Visualization")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")
        plt.show()
        return plt.gcf()

    def train_model(self, penalty, l1_ratio, c_value, max_iter):
        l1_ratio = l1_ratio if penalty == 'elasticnet' else None
        self.model = LogisticRegression(penalty=penalty, solver='saga', l1_ratio=l1_ratio, C=c_value, max_iter=max_iter)
        self.model.fit(self.X_train, self.y_train)

        y_pred = self.model.predict(self.X_test)
        acc = accuracy_score(self.y_test, y_pred)

        fig, ax = plt.subplots(figsize=(6, 5))
        sns.heatmap(confusion_matrix(self.y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")

        return acc, fig, classification_report(self.y_test, y_pred)

    def plot_decision_boundary(self):
        if self.model is None:
            return "Model is not trained yet!"

        x_min, x_max = self.X_train[:, 0].min() - 1, self.X_train[:, 0].max() + 1
        y_min, y_max = self.X_train[:, 1].min() - 1, self.X_train[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

        Z = self.model.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        fig, ax = plt.subplots()
        plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
        plt.scatter(self.X_train[:, 0], self.X_train[:, 1], c=self.y_train, edgecolor='k', cmap='coolwarm')
        plt.title("Decision Boundary")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")
        plt.show()
        return fig

# Gradio UI
def gradio_interface(X_train, X_test, y_train, y_test):
    tool = LogisticRegressionTool(X_train, X_test, y_train, y_test)

    return gr.Interface(
        fn=tool.train_model,
        inputs=[
            gr.Radio(["none", "l2", "l1", "elasticnet"], label="Penalty", value="l2"),
            gr.Slider(0, 1, 0.5, label="L1 Ratio (Elastic Net only)"),
            gr.Slider(0.01, 10, 1.0, label="Regularization Strength (C)"),
            gr.Slider(100, 5000, 1000, step=100, label="Max Iterations")
        ],
        outputs=["number", "plot", "text"],
        title="Logistic Regression Hyperparameter Tool",
        description="Select hyperparameters, train the model, and view accuracy, confusion matrix, and classification report."
    )

# Example usage (User should pass their own X_train, X_test, y_train, y_test)



In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
interface = gradio_interface(X_train, X_test, y_train, y_test)
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1be08d838c56607b33.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
